# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References


---

In [4]:
! pip install scikit-learn==1.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.16.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.


In [5]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Robust linear regression - Huber loss

In [19]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [29]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations

    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - epsilon: float
    - alpha: regularization
    """

    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0

    for  i in range(len(y)):
        diff = (w.T).dot(X[i])+b-y[i]
        if abs(diff) <= epsilon:
            loss+=(1/2)*((w.T).dot(X[i])+b - y[i])**2
        else:
            loss+=epsilon*abs(diff)-(epsilon**2)/2

    loss*=1/len(y)

    #dw

    for k in range (len(w)):
        for i in range (len (y)):
            diff = (w.T).dot(X[i])+b-y[i]
            if abs(diff) <= epsilon:
                dw[k] += X[i][k]*((w.T).dot(X[i])+b - y[i])
            else :
                dw[k] += X[i][k]* epsilon* np.sign(((w.T).dot(X[i])+b - y[i]))
    dw *=1/len(y)
    #db
    for i in range(len(y)):
      diff = (w.T).dot(X[i])+b-y[i]
      if abs(diff) <= epsilon:
        db += diff
      else :
        db += epsilon* np.sign(((w.T).dot(X[i])+b-y[i]))
    db*=1/len(y)

    loss+= alpha*(w @ w) + alpha *b*b
    dw += 2 * alpha * w
    db +=  2 * alpha *b

    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [30]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -0.748838 analytic: -0.748838, relative error: 5.742450e-10
numerical: -4.878257 analytic: -4.878257, relative error: 9.608527e-11
numerical: -551.120158 analytic: -551.120158, relative error: 9.650960e-13
numerical: -551.120158 analytic: -551.120158, relative error: 9.650960e-13
numerical: -12.891700 analytic: -12.891700, relative error: 6.087044e-12
numerical: -15.340909 analytic: -15.340909, relative error: 9.718030e-12
numerical: -0.748838 analytic: -0.748838, relative error: 5.742450e-10
numerical: -481.509943 analytic: -481.509943, relative error: 6.280389e-14
numerical: -0.748838 analytic: -0.748838, relative error: 5.742450e-10
numerical: -0.748838 analytic: -0.748838, relative error: 5.742450e-10
numerical: -551.120158 analytic: -551.120158, relative error: 9.650960e-13
numerical: -15.034651 analytic: -15.034651, relative error: 7.298001e-11
numerical: -15.340909 analytic: -15.340909, relative error: 9.718030e-12
numerical: -551.120158 analytic: -55

 ## with regularization

In [31]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -5.123191 analytic: -5.123191, relative error: 9.705827e-11
numerical: -15.034639 analytic: -15.034639, relative error: 7.254527e-11
numerical: -15.340948 analytic: -15.340948, relative error: 7.864351e-12
numerical: -12.891966 analytic: -12.891966, relative error: 4.065638e-12
numerical: -0.093181 analytic: -0.093181, relative error: 1.307923e-10
numerical: -17.081496 analytic: -17.081496, relative error: 3.050012e-12
numerical: -15.034639 analytic: -15.034639, relative error: 7.254527e-11
numerical: -15.340948 analytic: -15.340948, relative error: 7.864351e-12
numerical: -24.915267 analytic: -24.915267, relative error: 1.953936e-12
numerical: -5.123191 analytic: -5.123191, relative error: 9.705827e-11
numerical: -17.081496 analytic: -17.081496, relative error: 3.050012e-12
numerical: -17.081496 analytic: -17.081496, relative error: 3.050012e-12
numerical: -0.093181 analytic: -0.093181, relative error: 1.307923e-10
numerical: -92.576131 analytic: -92.576131

In [32]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations

    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0


    diff= X.dot(w) + b - y

    loss = np.where(np.abs(diff), 1/2 * (diff)**2, epsilon*np.abs(diff) - epsilon**2 / 2)
    loss = loss.mean()

    dloss = np.where(np.abs(diff), diff, epsilon*np.sign(diff))
    db = dloss.mean()
    dw = X.T.dot(dloss) / X.shape[0]

    loss+= alpha*(w @ w) + alpha *b*b
    dw += 2 * alpha * w
    db +=  2 * alpha *b
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [33]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -184.159159 analytic: -184.159159, relative error: 3.277092e-12
numerical: -50.621816 analytic: -50.621816, relative error: 3.284914e-11
numerical: -50.621816 analytic: -50.621816, relative error: 3.284914e-11
numerical: -1443.451143 analytic: -1443.451143, relative error: 5.003650e-13
numerical: -332.348005 analytic: -332.348005, relative error: 1.710529e-12
numerical: -1.962288 analytic: -1.962288, relative error: 7.742977e-10
numerical: -219.814175 analytic: -219.814175, relative error: 4.282376e-12
numerical: -12.008686 analytic: -12.008686, relative error: 1.154194e-10
numerical: -8290.524561 analytic: -8290.524561, relative error: 2.598862e-13
numerical: -1.962288 analytic: -1.962288, relative error: 7.742977e-10
numerical: -8449.099367 analytic: -8449.099367, relative error: 4.241173e-14
numerical: -184.159159 analytic: -184.159159, relative error: 3.277092e-12
numerical: -404.549691 analytic: -404.549691, relative error: 1.800078e-12
numerical: -1443

## with regularization

In [34]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -219.814163 analytic: -219.814163, relative error: 3.908056e-12
numerical: -8290.524118 analytic: -8290.524118, relative error: 3.082652e-13
numerical: -50.622006 analytic: -50.622006, relative error: 2.969921e-11
numerical: -50.622006 analytic: -50.622006, relative error: 2.969921e-11
numerical: -219.814163 analytic: -219.814163, relative error: 3.908056e-12
numerical: -145.674197 analytic: -145.674197, relative error: 3.332485e-12
numerical: -12.008925 analytic: -12.008925, relative error: 1.058132e-10
numerical: -1.962089 analytic: -1.962089, relative error: 6.157627e-10
numerical: -50.622006 analytic: -50.622006, relative error: 2.969921e-11
numerical: -12.008925 analytic: -12.008925, relative error: 1.058132e-10
numerical: -8290.524118 analytic: -8290.524118, relative error: 3.082652e-13
numerical: -12.008925 analytic: -12.008925, relative error: 1.058132e-10
numerical: -8449.099409 analytic: -8449.099409, relative error: 1.083975e-13
numerical: -145.67

In [37]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape

        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w

        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
             # YOUR CODE HERE
            index = np.random.choice(N,batch_size,replace=False)
            X_batch = np.array([X[i] for i in index])
            y_batch = np.array([y[i] for i in index])



            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update
            # Update the weights w and bias b using the gradient and the learning rate.
            # YOUR CODE HERE
            self.w= self.w- learning_rate * dw
            self.b = self.b- learning_rate * db



            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)

    def predict(self, X):
        # YOUR CODE HERE
        y = X.dot(self.w) + self.b
        return y


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

iteration 0 / 75000: loss 284.836255
iteration 10000 / 75000: loss 7.256567
iteration 20000 / 75000: loss 7.165263
iteration 30000 / 75000: loss 10.268618
iteration 40000 / 75000: loss 11.793308
iteration 50000 / 75000: loss 8.294548
iteration 60000 / 75000: loss 8.214502
iteration 70000 / 75000: loss 13.435476
MSE scikit-learn: 24.04102301055778
MSE gradient descent model : 21.918054529522752


# Multinomial logistic regression

In [39]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

In [42]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """

    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)

    # Tandremo ny numeric instability
    # YOUR CODE HERE
    W=W.T
    X=X.T
    dW = dW.T
    num_classes = W.shape[0]
    num_train = X.shape[1]

    for i in range(num_train):
        # Compute vector of scores
        f_i = W.dot(X[:, i]) # in R^{num_classes}

        # Normalization trick to avoid numerical instability, per http://cs231n.github.io/linear-classify/#softmax
        log_c = np.max(f_i)
        f_i -= log_c

        # Compute loss (and add to it, divided later)
        # L_i = - f(x_i)_{y_i} + log \sum_j e^{f(x_i)_j}
        sum_i = 0.0
        for f_i_j in f_i:
            sum_i += np.exp(f_i_j)
        loss += -f_i[y[i]] + np.log(sum_i)

        # Compute gradient
        # dw_j = 1/num_train * \sum_i[x_i * (p(y_i = j)-Ind{y_i = j} )]
        # Here we are computing the contribution to the inner sum for a given i.
        for j in range(num_classes):
            p = np.exp(f_i[j])/sum_i
            dW[j, :] += (p-(j == y[i])) * X[:, i]

    # Compute average
    loss /= num_train
    dW /= num_train

    # Regularization
    loss += 0.5 * alpha * np.sum(W * W)
    dW += alpha*W

    dW=dW.T

    return loss, dW



## Without regularization

In [43]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.275657 analytic: -0.275657, relative error: 6.064547e-11
numerical: 0.318071 analytic: 0.318071, relative error: 4.178127e-11
numerical: -0.031708 analytic: -0.031708, relative error: 3.978940e-09
numerical: -0.598316 analytic: -0.598316, relative error: 1.108522e-10
numerical: 0.318071 analytic: 0.318071, relative error: 4.178127e-11
numerical: 0.280437 analytic: 0.280437, relative error: 5.015025e-10
numerical: 0.027505 analytic: 0.027505, relative error: 8.944895e-10
numerical: -0.031708 analytic: -0.031708, relative error: 3.978940e-09
numerical: -0.598316 analytic: -0.598316, relative error: 1.108522e-10
numerical: 0.318071 analytic: 0.318071, relative error: 4.178127e-11
numerical: -0.598316 analytic: -0.598316, relative error: 1.108522e-10
numerical: -0.031708 analytic: -0.031708, relative error: 3.978940e-09


## With regularization

In [44]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.598189 analytic: -0.598189, relative error: 1.130301e-10
numerical: 0.280336 analytic: 0.280336, relative error: 4.839148e-10
numerical: -0.168128 analytic: -0.168128, relative error: 3.210746e-10
numerical: -0.248875 analytic: -0.248875, relative error: 6.231874e-10
numerical: 0.095317 analytic: 0.095317, relative error: 1.414266e-10
numerical: 0.280336 analytic: 0.280336, relative error: 4.839148e-10
numerical: 0.095317 analytic: 0.095317, relative error: 1.414266e-10
numerical: -0.275813 analytic: -0.275813, relative error: 6.996702e-11
numerical: -0.122599 analytic: -0.122599, relative error: 1.279049e-10
numerical: -0.031794 analytic: -0.031794, relative error: 3.870524e-09
numerical: -0.275813 analytic: -0.275813, relative error: 6.996702e-11
numerical: 0.095317 analytic: 0.095317, relative error: 1.414266e-10


In [45]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    # YOUR CODE HERE
    num_train = X.shape[0]
    f = X.dot(W)
    f = f - np.max(f, axis=1)[:, np.newaxis]
    loss = -np.sum(np.log(np.exp(f[np.arange(num_train), y]) / np.sum(np.exp(f), axis=1)))
    loss /= num_train
    loss += 0.5 * alpha * np.sum(W * W)
    ind = np.zeros_like(f)
    ind[np.arange(num_train), y] = 1
    dW = X.T.dot(np.exp(f) / np.sum(np.exp(f), axis=1, keepdims=True) - ind)
    dW /= num_train
    dW += alpha * W


    return loss, dW

## Without regularization

In [46]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.122878 analytic: -0.122878, relative error: 1.669786e-10
numerical: -0.275657 analytic: -0.275657, relative error: 1.990557e-11
numerical: -0.598316 analytic: -0.598316, relative error: 8.301847e-11
numerical: -0.275657 analytic: -0.275657, relative error: 1.990557e-11
numerical: -0.248730 analytic: -0.248730, relative error: 5.342213e-10
numerical: 0.280437 analytic: 0.280437, relative error: 4.619135e-10
numerical: -0.042414 analytic: -0.042414, relative error: 5.005900e-11
numerical: -0.042414 analytic: -0.042414, relative error: 5.005900e-11
numerical: 0.318071 analytic: 0.318071, relative error: 1.057590e-11
numerical: 0.318071 analytic: 0.318071, relative error: 1.057590e-11
numerical: -0.248730 analytic: -0.248730, relative error: 5.342213e-10
numerical: 0.280437 analytic: 0.280437, relative error: 4.619135e-10


## With regularization

In [47]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.766373 analytic: 0.766373, relative error: 7.360716e-11
numerical: 0.027384 analytic: 0.027384, relative error: 3.779838e-10
numerical: -0.031794 analytic: -0.031794, relative error: 4.045124e-09
numerical: -0.122599 analytic: -0.122599, relative error: 1.731833e-10
numerical: -0.122599 analytic: -0.122599, relative error: 1.731833e-10
numerical: -0.598189 analytic: -0.598189, relative error: 8.519046e-11
numerical: 0.766373 analytic: 0.766373, relative error: 7.360716e-11
numerical: 0.095317 analytic: 0.095317, relative error: 2.080031e-10
numerical: -0.275813 analytic: -0.275813, relative error: 1.053847e-11
numerical: -0.122599 analytic: -0.122599, relative error: 1.731833e-10
numerical: 0.318181 analytic: 0.318181, relative error: 6.465905e-12
numerical: 0.318181 analytic: 0.318181, relative error: 6.465905e-12


## Gradient descent

In [70]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # YOUR CODE HERE
            tmp = np.ones((len(X),1))
            X = np.append(tmp, X, axis = 1)

        N, d = X.shape

        C = (np.max(y) + 1)
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        C = (np.max(y) + 1)
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W

        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            index = np.random.choice(N,batch_size,replace=False)
            X_batch = np.array([X[i] for i in index])
            y_batch = np.array([y[i] for i in index])



            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update
            # Update the weights w using the gradient and the learning rate.
            # YOUR CODE HERE
            self.W= self.W- learning_rate * dW




            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)

    def predict(self, X):
        """
        Inputs:
        - X: array of shape (N, D)

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class
        """
        # YOUR CODE HERE
        if self.fit_intercept:
            # YOUR CODE HERE
            tmp = np.ones((len(X),1))
            X = np.append(tmp, X, axis=1)

        z = X.dot(self.W)
        y_pred = np.argmax(z, axis=1)

        return y_pred

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

iteration 0 / 75000: loss 1.098483
iteration 10000 / 75000: loss 0.402564
iteration 20000 / 75000: loss 0.423032
iteration 30000 / 75000: loss 0.334499
iteration 40000 / 75000: loss 0.393382
iteration 50000 / 75000: loss 0.295145
iteration 60000 / 75000: loss 0.337588
iteration 70000 / 75000: loss 0.356096
Accuracy scikit-learn: 0.86
Accuracy gradient descent model : 0.8666666666666667


In [71]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 1.099141
iteration 10000 / 75000: loss 0.337020
iteration 20000 / 75000: loss 0.258650
iteration 30000 / 75000: loss 0.221944
iteration 40000 / 75000: loss 0.207700
iteration 50000 / 75000: loss 0.201054
iteration 60000 / 75000: loss 0.177119
iteration 70000 / 75000: loss 0.139653
Accuracy scikit-learn: 0.9733333333333334
Accuracy gradient descent model : 0.96


# K-Nearest Neighbor

## Computing distances

In [75]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [76]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            # Ataovy ao anaty distances[i, j] ny distance entre ith test point sy th training point
            # Aza manao boucle instony ato anatiny
            # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            # YOUR CODE HERE
            distances[i][j] = np.sqrt(np.sum((X_test[i] - X_train[j])**2))

    return distances

In [74]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [79]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
        # Aza manao boucle instony ato anatiny
        # TSY MAHAZO MAMPIASA np.linalg.norm() :D

        # YOUR CODE HERE
         distances[i] = np.sqrt(np.sum((X_test[i] - X_train)**2, axis=1))

    return distances

In [80]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [81]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train

    Inputs:
    - X_test: array of shape (num_test, D)

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
    # Aza manao boucle instony
    # TSY MAHAZO MAMPIASA np.linalg.norm() NA FONCTIONS AO AMIN'NY SCIPY :D

    # YOUR CODE HERE
    M = X_test.dot(X_train.T)
    nrow = M.shape[0]
    ncol = M.shape[1]

    te = np.diag(X_test.dot(X_test.T))
    tr = np.diag(X_train.dot(X_train.T))

    te = np.reshape(np.repeat(te, ncol), M.shape)
    tr = np.reshape(np.repeat(tr, nrow), M.T.shape)

    sq = -2 * M + te + tr.T
    distances = np.sqrt(sq)

    return distances

In [82]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


## K-Nearest Neighbor (knn) classifier

In [83]:
class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D)
        - y: array of shape (N,)
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D)

        Returns:
        - y: array of shape (num_test,)
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,)
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            # Ampidirina ao anaty closest_y ny labels an'ny k neighbors akaiky indrindra
            # Jereo fampiasana np.argsort
            # YOUR CODE HERE
            r = np.argsort(distances[i, :])[:self.k]
            closest_y = self.y_train[r]



            # Tadiavo ny label betsaka indrindra dia iny no atao prediction
            # Raha misy mitovy dia izay label kely raisina
            # YOUR CODE HERE
            y_pred[i] = np.bincount(closest_y).argmax()


        return y_pred

In [84]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831


## cross-validation

In [85]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds

# YOUR CODE HERE
X_train, y_train = data.data, data.target

X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(y_train, num_folds)


# A dictionary of length num_folds holding the accuracies for different values of k
k_to_accuracies = {}

# Ataovy ary ilay k-fold cross validation
# Atao ao anaty k_to_accuracies ny accuracy isaky ny valeur k
# YOUR CODE HERE
for k in k_choices:
    model_cv = KNearestNeighborClassifier(k)
    k_to_accuracies[k] = np.zeros(num_folds)
    for i in range(num_folds):
        # test fold X_test_vc | y_train_cv
        X_test_cv = X_train_folds[i]
        y_test_cv = y_train_folds[i]

        # training folds X_train_cv | y_train_cv
        X_train_cv = None
        y_train_cv = None
        for j in range(num_folds):
            if i != j:
                if X_train_cv is None:
                    X_train_cv = X_train_folds[j]
                    y_train_cv = y_train_folds[j]
                else:
                    X_train_cv = np.append(X_train_cv, X_train_folds[j], axis=0)
                    y_train_cv = np.append(y_train_cv, y_train_folds[j], axis=0)

        # cross validation
        model_cv.fit(X_train_cv, y_train_cv)
        pred_cv = model_cv.predict(X_test_cv)

        # computing accuracy
        accuracy = accuracy_score(y_test_cv, pred_cv)
        k_to_accuracies[k][i] = accuracy

In [86]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

k = 1, accuracy = 0.961111
k = 1, accuracy = 0.952778
k = 1, accuracy = 0.966574
k = 1, accuracy = 0.988858
k = 1, accuracy = 0.955432
k = 3, accuracy = 0.955556
k = 3, accuracy = 0.961111
k = 3, accuracy = 0.963788
k = 3, accuracy = 0.986072
k = 3, accuracy = 0.966574
k = 5, accuracy = 0.950000
k = 5, accuracy = 0.963889
k = 5, accuracy = 0.963788
k = 5, accuracy = 0.980501
k = 5, accuracy = 0.963788
k = 8, accuracy = 0.941667
k = 8, accuracy = 0.961111
k = 8, accuracy = 0.966574
k = 8, accuracy = 0.974930
k = 8, accuracy = 0.949861
k = 10, accuracy = 0.938889
k = 10, accuracy = 0.952778
k = 10, accuracy = 0.966574
k = 10, accuracy = 0.974930
k = 10, accuracy = 0.949861
k = 12, accuracy = 0.941667
k = 12, accuracy = 0.955556
k = 12, accuracy = 0.966574
k = 12, accuracy = 0.974930
k = 12, accuracy = 0.949861
k = 15, accuracy = 0.941667
k = 15, accuracy = 0.955556
k = 15, accuracy = 0.966574
k = 15, accuracy = 0.972145
k = 15, accuracy = 0.947075
k = 20, accuracy = 0.930556
k = 20, accu